In [1]:
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from src.average_precision import calculate_delta
import pandas as pd
from src.average_precision import convert_to_names
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn import metrics
import xgboost
from xgboost.sklearn import XGBClassifier
%matplotlib inline
import seaborn
from sklearn.externals import joblib

In [2]:
X = pd.read_pickle("data/multiclass_training.pickle")

In [3]:
cols_not_for_training = ["fecha_dato","ncodpers", "target", "product"]
X_train, X_test, y_train, y_test = train_test_split(
                                    X.drop(cols_not_for_training, axis=1), X["target"], test_size=0.20, random_state=42)

In [6]:
model = XGBClassifier(
        objective = 'multi:softprob',
        nthread = 4,
        learning_rate = 0.05,
        n_estimators = 100,
        max_depth = 8,
        subsample = 0.7,
        colsample_bytree = 0.7,
        colsample_bylevel = 0.7,
        silent = 0,
        )

param_grid = {}
param_grid["colsample_bytree"] = [0.3, 0.7]

grid_search = GridSearchCV(model, param_grid, scoring="accuracy", cv=3, verbose=2)
cv_results = grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] colsample_bytree=0.3 ............................................


/home/valentin/bin/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV] ............................. colsample_bytree=0.3, total=  42.2s
[CV] colsample_bytree=0.3 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.5s remaining:    0.0s


[CV] ............................. colsample_bytree=0.3, total=  48.6s
[CV] colsample_bytree=0.3 ............................................
[CV] ............................. colsample_bytree=0.3, total=  32.1s
[CV] colsample_bytree=0.7 ............................................
[CV] ............................. colsample_bytree=0.7, total=  53.3s
[CV] colsample_bytree=0.7 ............................................
[CV] ............................. colsample_bytree=0.7, total=  54.0s
[CV] colsample_bytree=0.7 ............................................
[CV] ............................. colsample_bytree=0.7, total=  51.1s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  4.8min finished


In [ ]:
joblib.dump(cv_results, "models/xgboost_cv.pickle")